In [23]:
# Dependencies
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
from scipy.stats import uniform, expon
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
import pickle

Importing Steinmetz Data and K-Fold splits

In [24]:
df_tr = np.array(pd.read_csv("data_transformed_train.csv", index_col = 0))
splits = pd.read_csv('splits10by10.csv', index_col = 0).reset_index(drop = True) - 1

X_train = df_tr[:,0:20]
y_train = df_tr[:,20].reshape((X_train.shape[0],1))
print(X_train.shape, y_train.shape)

(7012, 20) (7012, 1)


In [25]:
df_te = np.array(pd.read_csv("data_transformed_test.csv", index_col = 0))

X_test = df_te[:,0:20]
y_test = df_te[:,20].reshape((X_test.shape[0],1))
print(X_test.shape, y_test.shape)

(1038, 20) (1038, 1)


Tuning Logistic Regression on Steinmetz Data

In [26]:
def inv_squared_diff(y_true, y_pred):
    """Computes the sum of squared differences for true and predicted labels.
       The score is negative to be maximized instead of minimized.

    Parameters
    ----------
    y_true : numpy.array
        The true labels
    y_pred : numpy.array
        The predicted label

    Returns
    -------
    float
        The negative of the sum of squared differences
    """
    return(-(0.5*np.sum((y_true - y_pred)**2)))

def save_object(obj, filename):
    """Saved python object as pickle file to current directory

    Parameters
    ----------
    obj : any
        The python object to save
    filename : str
        The file name to use for .pkl file (format: '_.pkl')

    Returns
    -------
    None
    """
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
        
def read_object(name):
    """Read pickle file as python object from current directory

    Parameters
    ----------
    name : str
        The .pkl file name excluding '.pkl' extension (format: name.pkl)

    Returns
    -------
    any
        The python object contained in the pickle file
    """
    fName = str(name + '.pkl')
    with open(fName, 'rb') as f:  # with statement avoids file leak
        return pickle.load(f)

In [27]:
params = [dict(lr__penalty = ['none'],
              lr__solver = ['newton-cg', 'lbfgs', 'sag', 'saga']), 
              dict(lr__penalty = ['l1'],
              lr__C = expon(0,7),
              lr__solver = ['liblinear', 'saga']),
              dict(lr__penalty = ['l2'],
              lr__C = expon(0,7),
              lr__solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
              dict(lr__penalty = ['elasticnet'],
              lr__l1_ratio = uniform(),
              lr__C = expon(0,7),
              lr__solver = ['saga'])]

scorer = make_scorer(inv_squared_diff, needs_proba = True) #Make common scorer to be maximized by search

searches = list()

n_iter = 3
max_epochs = 100
for i in range(n_iter):
    #Tune each model n_iter times using same folds as all other models.
    #Each fold is scaled before use
    #Initialize models differently each time but keep randomly sampled params constant
    print("======================")
    print("Iteration", str(i + 1))
    group_kfold = GroupKFold(n_splits=10)
    steps = [('scaler', StandardScaler(with_mean = False)), 
             ('lr', LogisticRegression(random_state = 123 + i))]
    pipeline = Pipeline(steps)

    lr = RandomizedSearchCV(pipeline, params, random_state = 123, return_train_score = True, cv = group_kfold, n_jobs = 2, verbose = 1)

    searches.append(lr.fit(X_train, np.ravel(y_train), groups = splits.iloc[:,i].tolist()))

print("======================")
print("====Done====")

save_object(searches, 'searches_lr.pkl')

Iteration 1
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Iteration 2
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Iteration 3
Fitting 10 folds for each of 10 candidates, totalling 100 fits
====Done====


In [28]:
print('Logistic Regression')
print(searches[0].best_params_)
print(searches[1].best_params_)
print(searches[2].best_params_)

Logistic Regression
{'lr__penalty': 'none', 'lr__solver': 'newton-cg'}
{'lr__penalty': 'none', 'lr__solver': 'newton-cg'}
{'lr__penalty': 'none', 'lr__solver': 'newton-cg'}


Training with best parameters

In [29]:
def get_classes(y_prob, threshold = .5):
    """Converts probability of positive class to binary 0,1 labels given a threshold
    
    Parameters
    ----------
    y_prob : numpy.array
        The predicted positive class probabilities
    threshold : float (default .5)
        The threshold above which a positive instance is predicted

    Returns
    -------
    numpy.array
        The binary class labels
    """
    return(np.array([1 if i > threshold else 0 for i in y_prob]))

def calc_metrics(y_true, y_pred):
    """Computes confusion matrix, accuracy, precision, recall, specificity, f1-score, and AUC for true and predicted labels

    Parameters
    ----------
    y_true : numpy.array
        The true labels
    y_pred : numpy.array
        The predicted label

    Returns
    -------
    tuple
        The first element is a confusion matrix, the second is a dictionary of remaining metrics
    """
    cm = confusion_matrix(np.ravel(y_true), y_pred.astype('int32'))
    acc = np.mean(y_true == y_pred.astype('int32'))
    auc = roc_auc_score(np.ravel(y_true), y_pred.astype('int32'))
    tn = cm[0,0]
    fn = cm[1,0]
    tp = cm[1,1]
    fp = cm[0,1]
    
    if(tp + fp == 0):
        precision = np.nan
    else:
        precision = tp/(tp + fp)
        
    if(tp + fn == 0):
        recall = np.nan
    else:
        recall = tp/(tp + fn)
    
    if(tn + fp == 0):
        specificity = np.nan
    else:
        specificity = tn/(tn + fp)
        
    if(precision + recall == 0):
        f1 = np.nan
    else:
        f1 = 2 * ((precision * recall)/(precision + recall))
    
    return((cm, {'Accuracy': [acc], 'AUC': [auc], 'Precision': [precision], 'Recall': [recall], 'Specificity': [specificity], 'F1-score': [f1]}))



In [30]:
errs_tr = list()
errs_val = list()

n_iter = 10
n_folds = 10
tuned_params = searches[0].best_params_

scores = pd.DataFrame()
cms = dict()
cms['LR'] = []

for i in range(n_iter):
    #Retrain models using best parameters on all training set and compute testing metrics n_iter times
    #Vary model initialization
    print("======================")
    print("Iteration", str(i + 1))
    groups = splits.iloc[:,i].tolist()
    group_kfold = GroupKFold(n_splits=n_folds)
    scaler = StandardScaler(with_mean = False)
    steps = [('scaler', scaler), 
           ('lr', LogisticRegression(penalty = tuned_params['lr__penalty'], 
                                     #C = tuned_params['lr__C'], 
                                     solver = tuned_params['lr__solver'], 
                                     #l1_ratio = tuned_params['lr__l1_ratio'], 
                                     random_state = 123 + i)
        )]
    
    #Initialize pipelines
    lr = Pipeline(steps)
    
    #Fit models on all training and validation data
    lr.fit(X_train, np.ravel(y_train))
    
    #Get predictions
    y_hat = lr.predict(X_test)
    
    #Get scores
    lr_cm, lr_scores = calc_metrics(np.ravel(y_test), y_hat)
    
    cms['LR'].append(lr_cm)
    
    scores = pd.concat([scores, pd.DataFrame(lr_scores)], axis = 0)
    
    for tr_index, val_index in group_kfold.split(X_train, y_train, groups):
        #Cross-validate error for models using best parameters
        #Vary model initialization at each outer iteration and training data at each inner iteration (fold)
        #Folds used are the same as all other models
        
        #Initialize pipelines
        lr = Pipeline(steps)
        
        #Fit models on training data only
        lr.fit(X_train[tr_index,:], np.ravel(y_train[tr_index,:]))
                
        #Errors/fold
        y_hat_tr = lr.predict_proba(X_train[tr_index,:])
        y_hat_val = lr.predict_proba(X_train[val_index,:])
        errs_tr.append(-inv_squared_diff(y_train[tr_index,:], y_hat_tr))
        errs_val.append(-inv_squared_diff(y_train[val_index,:], y_hat_val))

print("======================")
print("====Done====")

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
====Done====


In [31]:
#Get mean and sd of each testing metric (over 10 models)
scores_stats = scores.describe()
scores_stats_all = pd.concat([scores_stats.iloc[1,:], scores_stats.iloc[2,:]], axis = 1)
scores_stats_all.columns = ['LR_mean', 'LR_sd']

#Get errors of each model
errs = pd.DataFrame({'train_error': errs_tr, 'val_error': errs_val})

#Get mean and sd of errors (over 100 models)
errs_agg = errs.agg([np.nanmean, np.nanstd])

In [32]:
scores_stats_all

,LR_mean,LR_sd
Accuracy,0.597303,1.170278e-16
AUC,0.593626,0.000000e+00
Precision,0.660107,0.000000e+00
Recall,0.618090,1.170278e-16
Specificity,0.569161,0.000000e+00
F1-score,0.638408,1.170278e-16


In [33]:
errs_agg.T

,nanmean,nanstd
train_error,1835.254064,3.974300
val_error,203.926961,1.007372
